# Complete Code
```
import fasthtml.common as ft
import monsterui.all as mui
import os
import json
import glob

DATASET_DIR = os.path.join(os.path.dirname(__file__), "golden_dataset")

app, rt = mui.fast_app(hdrs=mui.Theme.blue.headers())

def list_traces():
    files = [f for f in os.listdir(DATASET_DIR) if f.endswith('.json')]
    files.sort()  # Changed to sort in ascending order
    items = []
    for fname in files:
        path = os.path.join(DATASET_DIR, fname)
        with open(path) as f:
            data = json.load(f)
        msg = data["request"]["messages"][0]["content"]
        dt = fname.split('_')[1] + ' ' + fname.split('_')[2]
        has_open_coding = bool(data.get("open_coding", ""))
        has_axial_coding = bool(data.get("axial_coding_code", ""))
        check_mark = "✅ " if has_open_coding else ""
        check_mark += "✅ " if has_axial_coding else ""
        items.append(
            ft.Li(ft.A(f"{check_mark}{dt}: {msg[:60]}...", href=annotate.to(fname=fname), cls=mui.AT.classic))
        )
    return ft.Ul(*items, cls=mui.ListT.bullet)

@rt
def index():
    return mui.Container(
        mui.H2("Golden Dataset Traces"),
        list_traces()
    )

def chat_bubble(m):
    is_user = m["role"] == "user"
    if m["role"] == "system":
        return ft.Details(
            ft.Summary("System Prompt"),
            ft.Div(
                mui.render_md(m["content"]),
                cls="chat-bubble chat-bubble-secondary"
            ),
            cls="chat chat-start"
        )
    return ft.Div(
        ft.Div(
            mui.render_md(m["content"]),
            cls=f"chat-bubble {'chat-bubble-primary' if is_user else 'chat-bubble-secondary'}"
        ),
        cls=f"chat {'chat-end' if is_user else 'chat-start'}"
    )

def get_unique_open_coding_codes():
    codes = set()
    for fname in glob.glob(os.path.join(DATASET_DIR, '*.json')):
        with open(fname) as f:
            data = json.load(f)
        note = data.get('open_coding', '')
        if note and note.strip().lower() != 'n/a':
            # Split on double space or newlines for multiple codes, else treat as one code
            for code in note.split('\n'):
                code = code.strip()
                if code:
                    codes.add(code)
    return sorted(codes)

def get_unique_axial_coding_codes():
    codes = set()
    for fname in glob.glob(os.path.join(DATASET_DIR, '*.json')):
        with open(fname) as f:
            data = json.load(f)
        code = data.get('axial_coding_code', '')
        if code and code.strip():
            codes.add(code.strip())
    return sorted(codes)

@rt
def annotate(fname:str):
    path = os.path.join(DATASET_DIR, fname)
    with open(path) as f:
        data = json.load(f)
    chat =  data["response"]["messages"]
    bubbles = [chat_bubble(m) for m in chat]
    notes = data.get("open_coding", "")
    axial_code = data.get("axial_coding_code", "")
    axial_code_options = get_unique_axial_coding_codes()
    # Get next and previous files
    files = [f for f in os.listdir(DATASET_DIR) if f.endswith('.json')]
    files.sort()  # Changed to sort in ascending order
    current_idx = files.index(fname)
    next_file = files[current_idx + 1] if current_idx < len(files) - 1 else files[0]
    prev_file = files[current_idx - 1] if current_idx > 0 else files[-1]
    return mui.Container(
        mui.DivFullySpaced(
            ft.A("Previous", href=annotate.to(fname=prev_file), cls=mui.AT.classic),
            ft.A("Home", href=index.to(), cls=mui.AT.classic),
            ft.A("Next", href=annotate.to(fname=next_file), cls=mui.AT.classic),
            cls="my-4"
        ),
        mui.Grid(
            ft.Div(*bubbles),
            mui.Form(
                mui.Select(
                    *[ft.Option(code, value=code) for code in axial_code_options],
                    id="axial_coding_code", icon=True, insertable=True, multiple=False,
                    value=axial_code, placeholder="Select or add axial coding (failure mode)..."
                ),
                mui.TextArea(notes, name="notes", value=notes, rows=20),
                mui.Button("Save", type="submit"),
                action=save_annotation.to(fname=fname), method="post",
                cls='w-full flex flex-col gap-2'
            ),
        ),
    )

@rt
def save_annotation(fname:str, notes:str, axial_coding_code:str=None):
    path = os.path.join(DATASET_DIR, fname)
    with open(path) as f:
        data = json.load(f)
    data["open_coding"] = notes
    if axial_coding_code is not None:
        data["axial_coding_code"] = axial_coding_code
    with open(path, "w") as f:
        json.dump(data, f)
    return ft.Redirect(annotate.to(fname=fname))

@rt
def theme():
    return mui.ThemePicker()

ft.serve()

# Imports


In [ ]:
import fasthtml.common as ft, json
import monsterui.all as mui
from fastcore.all import *
from fasthtml.common import *
from monsterui.all import *

from fasthtml.jupyter import render_ft
render_ft()

# make some dummy data


In [ ]:
dataset_dir = Path("Golden_Data_Set")
dataset_dir.mkdir(exist_ok=True)

In [ ]:
trace_example = {
    "request": {"messages": [{"role": "user", "content": "Hey! Need help with a pasta dish for tonight's potluck dinner - something that takes about an hour to make and NO seafood please (allergies in the group). Any ideas?"}]},
    "response": {"messages": [
        {"role": "system", "content": "You are an expert chef recommending delicious and useful recipes. Present only one recipe at a time. If the user doesn't specify what ingredients they have available, assume only basic ingredients are available."},
        {"role": "user", "content": "Hey! Need help with a pasta dish for tonight's potluck dinner - something that takes about an hour to make and NO seafood please (allergies in the group). Any ideas?"},
        {"role": "assistant", "content": "Absolutely! I recommend making a creamy mushroom and spinach tortellini bake – flavorful, comforting, and perfect for sharing at a potluck. It should take about an hour from start to finish."}
    ]}
}

In [ ]:
for i in range(3): (dataset_dir / f"trace_{20241201}_{120000 + i*1000}.json").write_text(json.dumps(trace_example, indent=2))

In [ ]:
dataset_dir

Path('Golden_Data_Set')

In [ ]:
dataset_dir.ls()

(#3) [Path('Golden_Data_Set/trace_20241201_120000.json'),Path('Golden_Data_Set/trace_20241201_121000.json'),Path('Golden_Data_Set/trace_20241201_122000.json')]

In [ ]:
files = L(dataset_dir.glob("*.json"))
files


(#3) [Path('Golden_Data_Set/trace_20241201_120000.json'),Path('Golden_Data_Set/trace_20241201_121000.json'),Path('Golden_Data_Set/trace_20241201_122000.json')]

# list the dummy data



In [ ]:
def list_traces():
    files = L(dataset_dir.glob("*.json")).sorted()
    return Ul(*[Li(A(f"{f.stem}: {dict2obj(f.read_json()).request.messages[0].content[:60]}...", href=f"/annotate/{f.name}")) for f in files])
list_traces()


<div>
  <ul>
    <li>
<a href="/annotate/trace_20241201_120000.json">trace_20241201_120000: Hey! Need help with a pasta dish for tonight&#x27;s potluck dinne...</a>    </li>
    <li>
<a href="/annotate/trace_20241201_121000.json">trace_20241201_121000: Hey! Need help with a pasta dish for tonight&#x27;s potluck dinne...</a>    </li>
    <li>
<a href="/annotate/trace_20241201_122000.json">trace_20241201_122000: Hey! Need help with a pasta dish for tonight&#x27;s potluck dinne...</a>    </li>
  </ul>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
Details(Summary("Click to expand"), ("Hidden content"))

<div>
<details class="border border-secondary rounded-lg "><summary class="font-medium  p-3 hover:bg-secondary cursor-pointer ">Click to expand</summary>Hidden content</details><script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
show(render_md("**Bold text** and *italic*"))

# Apply HTML styling


In [ ]:
from IPython.display import HTML
HTML("""<style>
.chat-bubble { padding: 12px 16px; border-radius: 18px; margin: 4px 0; max-width: 70%; display: inline-block;}
.chat-bubble-primary { background-color: #007bff; color: white; }
.chat-bubble-secondary { background-color: #f1f3f4; color: black; }
.chat { display: flex; margin: 8px 0; }
.chat-start { justify-content: flex-start; }
.chat-end { justify-content: flex-end; }
</style>""")

# Final form


In [ ]:
DivFullySpaced(
    A("← Previous", href="/prev"),
    A("🏠 Home", href="/"),
    A("Next →", href="/next"),
    cls="my-4"
)



<div>
  <div class="flex justify-between items-center my-4">
<a href="/prev">← Previous</a><a href="/">🏠 Home</a><a href="/next">Next →</a>  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


# Chat


In [ ]:
Div(Div("User message", cls="chat-bubble chat-bubble-primary"), cls="chat chat-end")

<div>
  <div class="chat chat-end">
    <div class="chat-bubble chat-bubble-primary">User message</div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
Div(Div("Assistant  message", cls="chat-bubble chat-bubble-primary"), cls="chat chat-start")

<div>
  <div class="chat chat-start">
    <div class="chat-bubble chat-bubble-primary">Assistant  message</div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
def chat_bubble(m):
    is_user = m["role"] == "user"
    if m["role"] == "system":
        return Details(
            Summary("System Prompt"),
            Div(render_md(m["content"]), cls="chat-bubble chat-bubble-secondary"),
            cls="chat chat-start"
        )
    return Div(
        Div(render_md(m["content"]), cls=f"chat-bubble {'chat-bubble-primary' if is_user else 'chat-bubble-secondary'}"),
        cls=f"chat {'chat-end' if is_user else 'chat-start'}"
    )
data = dict2obj(trace_example)
bubbles = data.response.messages.map(chat_bubble)
Container(*bubbles)


<div>
  <div class="uk-container mt-5 uk-container-xl">
<details class="border border-secondary rounded-lg chat chat-start"><summary class="font-medium  p-3 hover:bg-secondary cursor-pointer ">System Prompt</summary>      <div class="chat-bubble chat-bubble-secondary"><p class="text-lg leading-relaxed mb-6">You are an expert chef recommending delicious and useful recipes. Present only one recipe at a time. If the user doesn't specify what ingredients they have available, assume only basic ingredients are available.</p>
</div>
</details>    <div class="chat chat-end">
      <div class="chat-bubble chat-bubble-primary"><p class="text-lg leading-relaxed mb-6">Hey! Need help with a pasta dish for tonight's potluck dinner - something that takes about an hour to make and NO seafood please (allergies in the group). Any ideas?</p>
</div>
    </div>
    <div class="chat chat-start">
      <div class="chat-bubble chat-bubble-secondary"><p class="text-lg leading-relaxed mb-6">Absolutely! I recommend making a creamy mushroom and spinach tortellini bake – flavorful, comforting, and perfect for sharing at a potluck. It should take about an hour from start to finish.</p>
</div>
    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


# drop down


In [ ]:
sample_codes = ['bug', 'feature', 'error']
Select(*[Option(code, value=code) for code in sample_codes], placeholder="Select option...")

<div>
  <div class="h-10">
<uk-select cls-custom="button: uk-input-fake dropdown: w-full" placeholder="Select option..." hx-trigger=" delay:100ms" hx-include="this"><select hidden><option value="bug">bug</option><option value="feature">feature</option><option value="error">error</option></select></uk-select>  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


# Form


In [ ]:
Form(
    Select(*[Option(code, value=code) for code in sample_codes], placeholder="Select issue type..."),
    TextArea("Sample annotation", rows=5),
    Button("💾 Save", type="submit")
)

<div>
<form enctype="multipart/form-data" class="space-y-3">    <div class="h-10">
<uk-select cls-custom="button: uk-input-fake dropdown: w-full" placeholder="Select issue type..." hx-trigger=" delay:100ms" hx-include="this"><select hidden><option value="bug">bug</option><option value="feature">feature</option><option value="error">error</option></select></uk-select>    </div>
<textarea rows="5" class="uk-textarea ">Sample annotation</textarea><button type="submit" class="uk-btn uk-btn-default">💾 Save</button></form><script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
sample_bubbles = [
    Div(Div("User: Hello!", cls="chat-bubble chat-bubble-primary"), cls="chat chat-end"),
    Div(Div("Assistant: Hi there!", cls="chat-bubble chat-bubble-secondary"), cls="chat chat-start")
]

Container(
    DivFullySpaced(
        A("← Previous", href="/prev"),
        A("🏠 Home", href="/"),
        A("Next →", href="/next"),
        cls="my-4"
    ),
    Grid(
        Div(*sample_bubbles),
        Form(
            Select(*[Option(code, value=code) for code in sample_codes], placeholder="Select issue type..."),
            TextArea("Sample annotation", rows=5),
            Button("💾 Save", type="submit")
        )
    )
)

<div>
  <div class="uk-container mt-5 uk-container-xl">
    <div class="flex justify-between items-center my-4">
<a href="/prev">← Previous</a><a href="/">🏠 Home</a><a href="/next">Next →</a>    </div>
    <div class="grid grid-cols-1 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-2 xl:grid-cols-2 gap-4">
      <div>
        <div class="chat chat-end">
          <div class="chat-bubble chat-bubble-primary">User: Hello!</div>
        </div>
        <div class="chat chat-start">
          <div class="chat-bubble chat-bubble-secondary">Assistant: Hi there!</div>
        </div>
      </div>
<form enctype="multipart/form-data" class="space-y-3">        <div class="h-10">
<uk-select cls-custom="button: uk-input-fake dropdown: w-full" placeholder="Select issue type..." hx-trigger=" delay:100ms" hx-include="this"><select hidden><option value="bug">bug</option><option value="feature">feature</option><option value="error">error</option></select></uk-select>        </div>
<textarea rows="5" class="uk-textarea ">Sample annotation</textarea><button type="submit" class="uk-btn uk-btn-default">💾 Save</button></form>    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
files = L(globtastic(dataset_dir, file_glob='*.json')).map(lambda x: Path(x).name).sorted()
current_idx = files.index('trace_20241201_120000.json')
next_file = files[current_idx + 1] if current_idx < len(files) - 1 else files[0]
prev_file = files[current_idx - 1] if current_idx > 0 else files[-1]
print(f"Current: {current_idx}, Next: {next_file}, Prev: {prev_file}")

Current: 0, Next: trace_20241201_121000.json, Prev: trace_20241201_122000.json


In [ ]:
def get_unique_codes():
    codes = set()
    for f in dataset_dir.glob('*.json'):
        data = f.read_json()
        if 'axial_coding_code' in data and data['axial_coding_code'].strip():
            codes.add(data['axial_coding_code'].strip())
    return sorted(codes)

get_unique_codes()

[]

In [ ]:
def save_annotation(fname, notes, axial_coding_code=None):
    path = dataset_dir / fname
    data = path.read_json()
    data["open_coding"] = notes
    if axial_coding_code: data["axial_coding_code"] = axial_coding_code
    path.write_text(json.dumps(data, indent=2))
    return f"Saved annotations for {fname}"

save_annotation('trace_20241201_120000.json', 'Test annotation', 'bug')

'Saved annotations for trace_20241201_120000.json'

In [ ]:
def create_annotation_page(fname):
    data = (dataset_dir / fname).read_json()
    bubbles = L(data['response']['messages']).map(chat_bubble)
    return Container(
        DivFullySpaced(A("← Previous", href="/prev"), A("🏠 Home", href="/"), A("Next →", href="/next")),
        Grid(Div(*bubbles), Form(Select(*[Option(c, value=c) for c in get_unique_codes()], placeholder="Select code..."), TextArea("", rows=10), Button("💾 Save")))
    )

create_annotation_page('trace_20241201_120000.json')

<div>
  <div class="uk-container mt-5 uk-container-xl">
    <div class="flex justify-between items-center w-full">
<a href="/prev">← Previous</a><a href="/">🏠 Home</a><a href="/next">Next →</a>    </div>
    <div class="grid grid-cols-1 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-2 xl:grid-cols-2 gap-4">
      <div>
<details class="border border-secondary rounded-lg chat chat-start"><summary class="font-medium  p-3 hover:bg-secondary cursor-pointer ">System Prompt</summary>          <div class="chat-bubble chat-bubble-secondary"><p class="text-lg leading-relaxed mb-6">You are an expert chef recommending delicious and useful recipes. Present only one recipe at a time. If the user doesn't specify what ingredients they have available, assume only basic ingredients are available.</p>
</div>
</details>        <div class="chat chat-end">
          <div class="chat-bubble chat-bubble-primary"><p class="text-lg leading-relaxed mb-6">Hey! Need help with a pasta dish for tonight's potluck dinner - something that takes about an hour to make and NO seafood please (allergies in the group). Any ideas?</p>
</div>
        </div>
        <div class="chat chat-start">
          <div class="chat-bubble chat-bubble-secondary"><p class="text-lg leading-relaxed mb-6">Absolutely! I recommend making a creamy mushroom and spinach tortellini bake – flavorful, comforting, and perfect for sharing at a potluck. It should take about an hour from start to finish.</p>
</div>
        </div>
      </div>
<form enctype="multipart/form-data" class="space-y-3">        <div class="h-10">
<uk-select cls-custom="button: uk-input-fake dropdown: w-full" placeholder="Select code..." hx-trigger=" delay:100ms" hx-include="this"><select hidden><option value="bug">bug</option></select></uk-select>        </div>
<textarea rows="10" class="uk-textarea "></textarea><button type="submit" class="uk-btn uk-btn-default">💾 Save</button></form>    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
def has_annotations(fname):
    data = (dataset_dir / fname).read_json()
    has_open = bool(data.get("open_coding", "").strip())
    has_axial = bool(data.get("axial_coding_code", "").strip())
    return has_open, has_axial

has_annotations('trace_20241201_120000.json')

(True, True)

In [ ]:
def enhanced_list_traces():
    files = L(dataset_dir.glob("*.json")).sorted()
    items = []
    for f in files:
        data = f.read_json()
        msg = data["request"]["messages"][0]["content"]
        has_open, has_axial = has_annotations(f.name)
        check_mark = ("✅ " if has_open else "") + ("✅ " if has_axial else "")
        items.append(Li(A(f"{check_mark}{f.stem}: {msg[:60]}...", href=f"/annotate/{f.name}")))
    return Ul(*items)

enhanced_list_traces()

<div>
  <ul>
    <li>
<a href="/annotate/trace_20241201_120000.json">✅ ✅ trace_20241201_120000: Hey! Need help with a pasta dish for tonight&#x27;s potluck dinne...</a>    </li>
    <li>
<a href="/annotate/trace_20241201_121000.json">trace_20241201_121000: Hey! Need help with a pasta dish for tonight&#x27;s potluck dinne...</a>    </li>
    <li>
<a href="/annotate/trace_20241201_122000.json">trace_20241201_122000: Hey! Need help with a pasta dish for tonight&#x27;s potluck dinne...</a>    </li>
  </ul>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [ ]:
def create_full_annotation_page(fname):
    data = (dataset_dir / fname).read_json()
    bubbles = L(data['response']['messages']).map(chat_bubble)
    existing_notes = data.get("open_coding", "")
    existing_code = data.get("axial_coding_code", "")
    codes = get_unique_codes()
    return Container(
        DivFullySpaced(A("← Previous"), A("🏠 Home"), A("Next →")),
        Grid(
            Div(*bubbles),
            Form(
                Select(*[Option(c, value=c, selected=c==existing_code) for c in codes], placeholder="Select code..."),
                TextArea(existing_notes, rows=10),
                Button("💾 Save")
            )
        )
    )

create_full_annotation_page('trace_20241201_120000.json')

<div>
  <div class="uk-container mt-5 uk-container-xl">
    <div class="flex justify-between items-center w-full">
<a href="#">← Previous</a><a href="#">🏠 Home</a><a href="#">Next →</a>    </div>
    <div class="grid grid-cols-1 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-2 xl:grid-cols-2 gap-4">
      <div>
<details class="border border-secondary rounded-lg chat chat-start"><summary class="font-medium  p-3 hover:bg-secondary cursor-pointer ">System Prompt</summary>          <div class="chat-bubble chat-bubble-secondary"><p class="text-lg leading-relaxed mb-6">You are an expert chef recommending delicious and useful recipes. Present only one recipe at a time. If the user doesn't specify what ingredients they have available, assume only basic ingredients are available.</p>
</div>
</details>        <div class="chat chat-end">
          <div class="chat-bubble chat-bubble-primary"><p class="text-lg leading-relaxed mb-6">Hey! Need help with a pasta dish for tonight's potluck dinner - something that takes about an hour to make and NO seafood please (allergies in the group). Any ideas?</p>
</div>
        </div>
        <div class="chat chat-start">
          <div class="chat-bubble chat-bubble-secondary"><p class="text-lg leading-relaxed mb-6">Absolutely! I recommend making a creamy mushroom and spinach tortellini bake – flavorful, comforting, and perfect for sharing at a potluck. It should take about an hour from start to finish.</p>
</div>
        </div>
      </div>
<form enctype="multipart/form-data" class="space-y-3">        <div class="h-10">
<uk-select cls-custom="button: uk-input-fake dropdown: w-full" placeholder="Select code..." hx-trigger=" delay:100ms" hx-include="this"><select hidden></select></uk-select>        </div>
<textarea rows="10" class="uk-textarea "></textarea><button type="submit" class="uk-btn uk-btn-default">💾 Save</button></form>    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>
